In [16]:
from random import randint
from sys import getsizeof
from multiprocessing import Process
import os
from datetime import datetime

In [10]:
def create_data(filename='data.txt', size=17179869184):
    """
    Генерация чисел и запись в файл.
    filename - имя файла;
    size - минимальный размер файла.
    """
    length = 0
    with open(filename, 'wt') as file:
        while length < size:
            n = randint(0, 4294967295)
            s = f'{n}\n'
            file.write(s)
            length += getsizeof(s)

In [12]:
def read_data(filename='data.txt'):
    """
    Построчное чтение чисел из файла.
    Возвращает список из прочитанных чисел.
    """
    with open(filename, 'rt') as file:
        data = []
        for line in file:
            data.append(int(line))
    return data

In [13]:
def write_to_file(data, i):
    """
    Запись в файл минимального и максимального чисел из полученного списка, а также суммы всех чисел.
    """
    with open('res%i' %i, 'wt') as file:
        file.write("%i\n%i\n%i" %
                   (min(data), max(data), sum(data)))

In [14]:
def get_part(data, num, part):
    """
    Разбиение списка data на num частей, возврат части с номером part (нумерация с единицы).
    """
    avg = len(data) / float(num)
    start = avg*(part-1)
    stop = avg*part
    return data[int(start):int(stop)]

In [15]:
def process_data(data, mode='single'):
    """
    Поиск минимального и максимального числа в наборе data, нахождение суммы всех чисел.
    data - набор чисел;
    mode - режим работы.
    Режимы работы: single, fork, multiprocessing.
    """
    if mode == 'single':
        return min(data), max(data), sum(data)
    else:
        min_int, max_int, sum_int = None, None, None
        size = len(data)//6
        children = []
        for i in range(1, 7):
            if mode == 'fork':
                p = os.fork()
                if not p:
                    write_to_file(get_part(data, 6, i), i)
                    os.abort()
                else:
                    children.append(int(p))
            elif mode == 'multiprocessing':
                children.append(Process(target=write_to_file, args=(get_part(data, 6, i), i)))
                children[-1].start()
        for p in children:
            if mode == 'fork':
                os.waitpid(p, 0)
            elif mode == 'multiprocessing':
                p.join()
        min_int, max_int, sum_int = [], [], []
        for i in range(1, 7):
            with open('res%i' %i, 'rt') as file:
                min_int.append(int(file.readline()))
                max_int.append(int(file.readline()))
                sum_int.append(int(file.readline()))
    return min(min_int), max(max_int), sum(sum_int)

In [ ]:
timestamp = datetime.now()
create_data()
print(f"Creating data: {datetime.now()-timestamp}")
timestamp = datetime.now()
data = read_data()
print(f"Reading data: {datetime.now()-timestamp}\n")
modes = ['single', 'fork', 'multiprocessing']
for mode in modes:
    timestamp = datetime.now()
    result = process_data(data, mode)
    print(f"Mode: {mode}, time: {datetime.now()-timestamp},\n{result}\n")

Creating data: 0:27:28.044184


In [8]:
process_data(data, mode='single')

(135233364, 3960806896, 45330325847)

In [9]:
process_data(data, mode='multiprocessing')

(135233364, 3960806896, 45330325847)